# Read-Copy Update (RCU)

1. [RCU OVERVIEW](https://www.kernel.org/doc/html/latest/RCU/whatisRCU.html)

The basic idea behind RCU is to split updates into “removal” and “reclamation” phases. The removal phase removes references to data items within a data structure (possibly by replacing them with references to new versions of these data items), and can run concurrently with readers. The reason that it is safe to run the removal phase concurrently with readers is the semantics of modern CPUs guarantee that readers will see either the old or the new version of the data structure rather than a partially updated reference. The reclamation phase does the work of reclaiming (e.g., freeing) the data items removed from the data structure during the removal phase. Because reclaiming data items can disrupt any readers concurrently referencing those data items, the reclamation phase must not start until readers no longer hold references to those data items.

Splitting the update into removal and reclamation phases permits the updater to perform the removal phase immediately, and to defer the reclamation phase until all readers active during the removal phase have completed, either by blocking until they finish or by registering a callback that is invoked after they finish. Only readers that are active during the removal phase need be considered, because any reader starting after the removal phase will be unable to gain a reference to the removed data items, and therefore cannot be disrupted by the reclamation phase.

So the typical RCU update sequence goes something like the following:

Remove pointers to a data structure, so that subsequent readers cannot gain a reference to it.

Wait for all previous readers to complete their RCU read-side critical sections.

At this point, there cannot be any readers who hold references to the data structure, so it now may safely be reclaimed (e.g., kfree()d).

Step (b) above is the key idea underlying RCU’s deferred destruction. The ability to wait until all readers are done allows RCU readers to use much lighter-weight synchronization, in some cases, absolutely no synchronization at all. In contrast, in more conventional lock-based schemes, readers must use heavy-weight synchronization in order to prevent an updater from deleting the data structure out from under them. This is because lock-based updaters typically update data items in place, and must therefore exclude readers. In contrast, RCU-based updaters typically take advantage of the fact that writes to single aligned pointers are atomic on modern CPUs, allowing atomic insertion, removal, and replacement of data items in a linked structure without disrupting readers. Concurrent RCU readers can then continue accessing the old versions, and can dispense with the atomic operations, memory barriers, and communications cache misses that are so expensive on present-day SMP computer systems, even in absence of lock contention.

In the three-step procedure shown above, the updater is performing both the removal and the reclamation step, but it is often helpful for an entirely different thread to do the reclamation, as is in fact the case in the Linux kernel’s directory-entry cache (dcache). Even if the same thread performs both the update step (step (a) above) and the reclamation step (step (c) above), it is often helpful to think of them separately. For example, RCU readers and updaters need not communicate at all, but RCU provides implicit low-overhead communication between readers and reclaimers, namely, in step (b) above.

So how the heck can a reclaimer tell when a reader is done, given that readers are not doing any sort of synchronization operations??? Read on to learn about how RCU’s API makes this easy.


2. RCU读取侧进入临界区的标志是调用rcu_read_lock，这个函数的代码是：
```c
#include <include/linux/rcupdate.h>
static inline void rcu_read_lock(void)
{
    __rcu_read_lock();
    __acquire(RCU)；
    rcu_read_acquire();
}
```

该实现里面貌似有三个函数调用，但实质性的工作由第一个函数__rcu_read_lock()来完成，__rcu_read_lock()通过调用 preempt_disable()关闭内核可抢占性。但是中断是允许的，假设读取者正处于rcu临界区中且刚读取了一个共享数据区的指针p(但是还没有访问p中的数据成员)，发生了一个中断，而该中断处理例程ISR恰好需要修改p所指向的数据区，按照RCU的设计原则，ISR会新分配一个同样大小的数据区new_p，再把老数据区p中的数据拷贝到新数据区，接着是在new_p的基础上做数据修改的工作(因为是在new_p空间中修改，所以不存在对p的并发访问，因此说RCU是一种免锁机制，原因就在这里)，ISR在把数据更新的工作完成后，将new_p赋值给p(p=new_p),最后它会再注册一个回调函数用以在适当的时候释放老指针p。因此，只要对老指针p上的所有引用都结束了，释放p就不会有问题。当中断处理例程做完这些工作返回后，被中断的进程将依然访问到p空间上的数据，也就是老数据，这样的结果是RCU机制所允许的。RCU规则对读取者与写入者之间因指针切换所造成的短暂的资源视图不一致问题是允许的。

接下来关于RCU一个有趣的问题是：何时才能释放老指针。我见过很多书中对此的回答是：当系统中所有处理器上都发生了一次进程切换。这种程式化的回答常常让刚接触RCU机制的读者感到一头雾水，为什么非要等所有处理器上都发生一次进程切换才可以调用回调函数释放老指针呢？这其实是RCU的设计规则决定的： 所有对老指针的引用只可能发生在rcu_read_lock与rcu_read_unlock所包括的临界区中，而在这个临界区中不可能发生进程切换，而一旦出了该临界区就不应该再有任何形式的对老指针p的引用。很明显，这个规则要求读取者在临界区中不能发生进程切换，因为一旦有进程切换，释放老指针的回调函数就有可能被调用，从而导致老指针被释放掉，当被切换掉的进程被重新调度运行时它就有可能引用到一个被释放掉的内存空间。

现在我们看到为什么rcu_read_lock只需要关闭内核可抢占性就可以了，因为它使得即便在临界区中发生了中断，当前进程也不可能被切换除去。 内核开发者，确切地说，RCU的设计者所能做的只能到这个程度。接下来就是使用者的责任了，如果在rcu的临界区中调用了一个函数，该函数可能睡眠，那么RCU的设计规则就遭到了破坏，系统将进入一种不稳定的状态。

这再次说明，如果想使用一个东西，一定要搞清楚其内在的机制，象上面刚提到的那个例子，即便现在程序不出现问题，但是系统中留下的隐患如同一个定时炸弹， 随时可能被引爆，尤其是过了很长时间问题才突然爆发出来。绝大多数情形下，找到问题所花费的时间可能要远远大于静下心来仔细搞懂RCU的原理要多得多。

RCU中的读取者相对rwlock的读取者而言，自由度更高。因为RCU的读取者在访问一个共享资源时，不需要考虑写入者的感受，这不同于rwlock的写入者，rwlock reader在读取共享资源时需要确保没有写入者在操作该资源。两者之间的差异化源自RCU对共享资源在读取者与写入者之间进行了分离，而rwlock的 读取者和写入者则至始至终只使用共享资源的一份拷贝。这也意味着RCU中的写入者要承担更多的责任，而且对同一共享资源进行更新的多个写入者之间必须引入某种互斥机制，所以RCU属于一种”免锁机制”的说法仅限于读取者与写入者之间。所以我们看到：RCU机制应该用在有大量的读取操作，而更新操作相对较少的情形下。此时RCU可以大大提升系统系能，因为RCU的读取操作相对其他一些有锁机制而言，在锁上的开销几乎没有。

实际使用中，共享的资源常常以链表的形式存在，内核为RCU模式下的链表操作实现了几个接口函数，读取者和使用者应该使用这些内核函数，比如 list_add_tail_rcu, list_add_rcu，hlist_replace_rcu等等，具体的使用可以参考某些内核编程或者设备驱动程序方面的资料。

在释放老指针方面，Linux内核提供两种方法供使用者使用，一个是调用call_rcu,另一个是调用synchronize_rcu。前者是一种异步 方式，call_rcu会将释放老指针的回调函数放入一个结点中，然后将该结点加入到当前正在运行call_rcu的处理器的本地链表中，在时钟中断的 softirq部分（RCU_SOFTIRQ）， rcu软中断处理函数rcu_process_callbacks会检查当前处理器是否经历了一个休眠期(quiescent，此处涉及内核进程调度等方面的内容)，rcu的内核代码实现在确定系统中所有的处理器都经历过了一个休眠期之后(意味着所有处理器上都发生了一次进程切换，因此老指针此时可以被安全释放掉了)，将调用call_rcu提供的回调函数。
synchronize_rcu的实现则利用了等待队列，在它的实现过程中也会向call_rcu那样向当前处理器的本地链表中加入一个结点，与 call_rcu不同之处在于该结点中的回调函数是wakeme_after_rcu，然后synchronize_rcu将在一个等待队列中睡眠，直到系统中所有处理器都发生了一次进程切换，因而wakeme_after_rcu被rcu_process_callbacks所调用以唤醒睡眠的 synchronize_rcu，被唤醒之后，synchronize_rcu知道它现在可以释放老指针了。

所以我们看到，call_rcu返回后其注册的回调函数可能还没被调用，因而也就意味着老指针还未被释放，而synchronize_rcu返回后老指针肯定被释放了。所以，是调用call_rcu还是synchronize_rcu，要视特定需求与当前上下文而定，比如中断处理的上下文肯定不能使用 synchronize_rcu函数了。

------------------------------------

## TODO: read code